In [ ]:
import pandas as pd
df = pd.read_csv("titanic.csv")

# 快速預覽資料（進入 GroupBy 前的暖身）

df.head()
df.info()
df.describe()
df.isnull().sum()

# 各艙等平均年齡、各艙等平均存活率
# 重新命名欄位,要注意原始欄位大小寫，否則會出現KeyError

df.groupby('pclass').agg(
    mean_age=('age', 'mean'),
    survival_rate=('survived', 'mean')
)

# 各艙等的平均年齡、年齡標準差，以及存活率
# 練習多層聚合，改進輸出結果可讀性

df.groupby('pclass').agg(
    mean_age=("age", "mean"),
    std_age=("age", "std"),
    survival_rate=("survived", "mean"))

In [ ]:
# 用樞紐表一次同時比較兩個分類變數對某個數值變數的影響，快速呈現分組之間的交互結果。
# 1. 匯入函式
import pandas as pd

# 2. 建立 pivot table
pivot = df.pivot_table(
    index= 'sex',
    columns='pclass',
    values='age',
    aggfunc='median'
)

# 3. 顯示結果
pivot

# 交叉分析存活率 & 排序篩選
# 1. 交叉計算存活率
pivot_sr = df.pivot_table(
    index='sex',
    columns='pclass',
    values='survived',
    aggfunc='mean'
)

# 2. 轉成長表方便排序
sr_long = pivot_sr.stack()  # 欄索引（pclass＝1、2、3）變成第二層行索引

# 3. 排序
sr_sorted = sr_long.sort_values(ascending=False) #由大到小排序

# 4. 看最前面（最高）和最後面（最低）幾筆
highest = sr_sorted.head(1)
lowest  = sr_sorted.tail(1)

pivot_sr


In [ ]:
# 匯入並預覽 flights 資料

import pandas as pd
df_f = pd.read_csv("flights.csv")
df_f.head()
df_f.info()

# 建立 DatetimeIndex
# 設計合成日期字串(年-月-日)
year_str = df_f['year'].astype(str)                 # year轉成字串
date_str = year_str + '-' + df_f['month'] + '-01'   # 連成日期字串
df_f['date_str'] = date_str                         # 新增date_str欄位
df_f['date'] = pd.to_datetime(df_f['date_str'])     # 字串轉成Datetime
df_f = df_f.set_index('date')                       # 設為索引
df_f.head()

# 重取樣到年度
# 年度總和：算出每一年所有月份的 passengers 總和
# 年度平均：算出每一年所有月份的 passengers 平均

#如果是 df_f.resample("YE").agg(...) → 需要寫 (欄位, 函式)
#如果是 df_f['passengers'].resample("YE").agg(...) → 直接寫函式名稱或用關鍵字參數命名
df_f['passengers'].resample("YE").agg(annual_passengers_sum="sum",
                                      annual_passengers_mean="mean",
                                      annual_std="std") 

In [ ]:
# 時間序列的平滑化
# 計算原始月度資料的：3/6/12 個月移動平均

avg_3 = df_f['passengers'].rolling(window=3).mean()
df_f['avg_3'] = avg_3
avg_6 = df_f['passengers'].rolling(window=6).mean()
df_f['avg_6'] = avg_6
avg_12 = df_f['passengers'].rolling(window=12).mean()
df_f['avg_12'] = avg_12

# 假設挑 1950-06-01 這一列,比較並檢查移動平均是否正確
check = df_f['passengers']['1950-04':'1950-06']
print("這三個月的值：\n", check)
print("手動平均：", check.mean())
print("rolling 平均：", df_f.loc['1950-06-01', 'avg_3'])

# 用Matplotlib畫線圖，觀察趨勢是否平滑化

import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(df_f.index, df_f['passengers'], label='Original')
plt.plot(df_f.index, df_f['avg_3'], label='3-month avg')
plt.plot(df_f.index, df_f['avg_6'], label='6-month avg')
plt.plot(df_f.index, df_f['avg_12'], label='12-month avg')

plt.title("Monthly Passengers and Moving Averages")
plt.xlabel("Date")
plt.ylabel("Passengers")
plt.legend()
plt.show()

# 結果
# window 越小（3 個月） → 線條接近原始資料，只是稍微平滑
# window 越大（12 個月）→ 線條更平滑，季節性波動幾乎被消除，只剩下長期趨勢

# 用seabor畫線圖，觀察趨勢是否平滑化

import seaborn as sns

# 加網格背景
sns.set_style("whitegrid")
plt.figure(figsize=(10,6))

# 自訂顏色
colors = ["blue", "orange", "green", "red"]

sns.lineplot(x=df_f.index, y=df_f['passengers'], label='Original', color=colors[0])
sns.lineplot(x=df_f.index, y=df_f['avg_3'], label='3-month avg', color=colors[1])
sns.lineplot(x=df_f.index, y=df_f['avg_6'], label='6-month avg', color=colors[2])
sns.lineplot(x=df_f.index, y=df_f['avg_12'], label='12-month avg', color=colors[3])

# 加上marker
sns.lineplot(x=df_f.index, y=df_f['passengers'], color=colors[0], marker="o", markevery=12, label=None)


plt.title("Monthly Passengers and Moving Averages")
plt.show()


## 資料視覺化統整的目標

1. 類別資料 → 長條圖 / 堆疊圖
- 例如：乘客性別分佈、各艙等存活率。
2. 數值資料 → 直方圖 / 箱型圖
- 例如：乘客年齡分佈、票價分佈。
3. 時間序列資料 → 折線圖
- 例如：機票乘客數變化、移動平均趨勢。

In [ ]:
# 資料視覺化統整
# 群組長條圖：性別與艙等存活率比較
# .reset_index() 把多層索引轉成一般欄位，方便繪圖
df_rate = df.groupby(['sex', 'pclass']).agg(survival_rate=("survived", "mean")).reset_index()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.barplot(data=df_rate, x="pclass", y="survival_rate", hue="sex")

plt.title("Survival Rate by Sex and Class")
plt.ylabel("Survival Rate")
plt.xlabel("Passenger Class")
plt.ylim(0, 1)  # 存活率介於 0~1
plt.show()

# 觀察結果：女性存活率遠高於男性，1 等艙的存活率最高，尤其是女性

# 直方圖/箱型圖：年齡分佈
# STEP 1 : 直方圖 (Histogram)

plt.figure(figsize=(8,5))
# bins=20 → 把年齡分成 20 個區間 | kde=True → 加上平滑曲線
sns.histplot(data=df, x="age", bins=20, kde=True) 

plt.title("Age Distribution of Passengers")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

# 觀察結果：乘客年齡主要集中在 20–30 歲之間，趨勢有右偏，少數人年齡超過 60 歲

# STEP 2 : 箱型圖 (Boxplot)

plt.figure(figsize=(8,5))
sns.boxplot(data=df, x="pclass", y="age")

plt.title("Age Distribution by Passenger Class")
plt.xlabel("Passenger Class")
plt.ylabel("Age")
plt.show()

# 年齡分佈分析
# 年齡主要集中在 20–30 歲之間，呈現右偏分佈。
# 一等艙乘客的年齡中位數最高，三等艙最低。
# 三等艙的乘客年齡範圍最廣，包含更多小孩與年長乘客。

# 直方圖/箱型圖：票價分佈
# STEP 1 : 直方圖 (Histogram)

plt.figure(figsize=(8,5))
# 對票價取對數，減少極端值影響
sns.histplot(data=df, x="fare", bins=30, log_scale=True)

plt.title("Fare Distribution of Passengers")
plt.xlabel("fare")
plt.ylabel("Count")
plt.show()

# STEP 2 : 箱型圖 (Boxplot)

plt.figure(figsize=(8,6))
sns.boxplot(data=df, x="pclass", y="fare")

plt.title("Fare Distribution by Passenger Class")
plt.ylabel("Fare")
plt.xlabel("Passenger Class")
plt.ylim(0, 300)  # 可限制範圍避免極端值影響
plt.show()

# 觀察
# 票價呈現長尾分佈，少數極高票價影響整體平均
# 艙等越高 → 票價越高且變異更大

# 折線圖：年度乘客總數趨勢

# 計算年度乘客總數
annual_sum = df_f['passengers'].resample("YE").sum()

plt.figure(figsize=(8,6))
sns.lineplot(x=annual_sum.index, y=annual_sum.values, marker="o")
plt.title("Annual Total Passengers")
plt.xlabel("Year")
plt.ylabel("Total Passengers")
plt.show()


# 類別資料分析 (value_counts、crosstab)。

## 學習目標

- 使用 value_counts() 觀察類別欄位的分佈
- 使用 pd.crosstab() 建立交叉表，分析兩個類別變數之間的關係
- 練習搭配 normalize 參數，計算百分比而不是計數

## 已知概念

- df['欄位'].value_counts() → 計算每個類別出現的次數
- df.groupby() → 也能算分組的統計
- crosstab 就像是 Pivot Table 的快速版本，但語法更簡潔